In [1]:
import pandas as pd
import requests
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

In [40]:
# 地区图鉴网址
original_url = 'https://wiki.52poke.com/wiki/%E5%AE%9D%E5%8F%AF%E6%A2%A6%E5%88%97%E8%A1%A8%EF%BC%88%E6%8C%89%E8%93%9D%E8%8E%93%E5%9B%BE%E9%89%B4%E7%BC%96%E5%8F%B7%EF%BC%89'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}
html = requests.get(original_url, headers=headers).text

In [41]:
# 获取信息列表
all_data = re.findall(r'<tr>(.*?<td>.*?</td>.*?<td>.*?</td>)</tr>', html, re.S)[1:]  # 去除第一个

# 爬取名称
name_lst = [re.findall(r'<td><a .*?title="(.*?)">', item)[-1]
            for item in all_data]

# 爬取编码信息（待转换）
numbers = [re.findall(r'<td>(#.*?)</td>', item, re.S) for item in all_data]
# 部分pokemon无旧地区编号，以空填充
# numbers = [item if len(item) != 2 else [''] + item for item in numbers]

# 爬取详细网址
url_lst = ['https://wiki.52poke.com/' +
           re.search(fr'<a href="(.*?)".*?title="{item}">.*?</a>', all_data[i]).group(1)
           for i, item in enumerate(name_lst)]

# 爬取属性(待转换)
tem_attributes = [re.findall(r'<a href=.*?（属性）">(.*?)</a>', item)
                  for item in all_data]

head_data = {
    '全国编号': [item[1].replace('\n', '') for item in numbers],
    '地区编号': [item[0].replace('\n', '') for item in numbers],
    '名称': name_lst,
    '属性': ['/'.join(item) for item in tem_attributes]
}
df = pd.DataFrame(head_data)

In [42]:
df

,全国编号,地区编号,名称,属性
0,#0084,#001,嘟嘟,一般/飞行
1,#0085,#002,嘟嘟利,一般/飞行
2,#0102,#003,蛋蛋,草/超能力
3,#0103,#004,椰蛋树,草/龙
4,#0111,#005,独角犀牛,地面/岩石
...,...,...,...,...
238,#1022,#239,铁磐岩,岩石/超能力
239,#1024,#240,太乐巴戈斯,一般
240,#1009,#241,波荡水,水/龙
241,#1010,#242,铁斑叶,草/超能力


In [43]:
# 爬取详细信息
def scrape_data(poke_url):
    poke_html = requests.get(poke_url, headers=headers).text

    # 筛选特性（普通和隐藏）
    character_text = re.search(r'>特性</a>(.*?)</tbody>', poke_html, re.S).group(1)  # 获取详细数据
    all_character_lst = re.findall(r'<a.*?title=.*?（特性）">(.*?)</a>', character_text)

    # 找出隐藏特性，若没有则None
    try:
        hidden_character_lst = re.findall(r'（特性）.*?</td>.*?title=.*?（特性）">(.*?)</a>.*?<small>', character_text, re.S)
    except AttributeError:
        hidden_character_lst = [None]

    # Pokemon普通与隐藏特性
    normal_character_lst = set(all_character_lst) - set(hidden_character_lst)  # 先移除隐藏特性
    normal_character = '/'.join(normal_character_lst)  # 普通
    hidden_character = '/'.join(hidden_character_lst)  # 隐藏

    # 暂存Pokemon的信息
    data = {
        '普通特性': normal_character,
        '隐藏特性': hidden_character,
        '类别': re.search(r'>分类<.*?title=.*?>(.*?)</a>', poke_html, re.S).group(1).strip(),
        '身高': re.search(r'>身高<.*?<td.*?>(.*?)</td>', poke_html, re.S).group(1).strip(),
        '体重': re.search(r'>体重<.*?<td.*?>(.*?)</td>', poke_html, re.S).group(1).strip(),
        '捕获率': re.search(r'>捕获率<.*?<td.*?>(.*?)<small>', poke_html, re.S).group(1).strip(),
        # HP, 攻击, 防御, 特攻, 特防, 速度, 总和
        '种族值': re.findall(r'<span style="float:right">(.*?)</span>', poke_html)[:7]
    }
    return data


# 爬取第二部分信息
# scrape_data(url_lst[120])
body_data = [scrape_data(url) for url in tqdm(url_lst)]

100%|██████████| 243/243 [02:24<00:00,  1.68it/s]


In [44]:
# 拼接数据
poke_data = pd.concat([df, pd.DataFrame(body_data)], axis=1)

# 保存为csv
poke_data.to_csv(r'd:\desktop\帕底亚_蓝莓学院图鉴.csv', encoding='utf-8-sig', index=False)